In [3]:
# Load the dataset
import pandas as pd

data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Display the first few rows and the data types
data.head(), data.dtypes, data.info(), data.describe()

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte

In [ ]:
# Adding lag features for New_Cases and New_Deaths
data['lag_1_cases'] = data['New_Cases'].shift(1)
data['lag_1_deaths'] = data['New_Deaths'].shift(1)

# Dropping any rows with NaN values created by lag
data.dropna(inplace=True)

# Adding additional time-based features
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day
data['DayOfWeek'] = data.index.dayofweek
data['WeekOfYear'] = data.index.isocalendar().week.astype(int)

# Display the preprocessed dataset and the newly added features
data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf

# Plotting correlations between features
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

# Plotting the autocorrelation of New_Cases
plt.figure(figsize=(10, 4))
plot_acf(data['New_Cases'], lags=50)
plt.title('Autocorrelation of New Cases')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.show()

# Plotting the autocorrelation of New_Deaths
plt.figure(figsize=(10, 4))
plot_acf(data['New_Deaths'], lags=50)
plt.title('Autocorrelation of New Deaths')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

# Plotting the partial autocorrelation of New_Cases
plt.figure(figsize=(10, 4))
plot_pacf(data['New_Cases'], lags=50, method='ywm')
plt.title('Partial Autocorrelation of New Cases')
plt.xlabel('Lags')
plt.ylabel('Partial Autocorrelation')
plt.show()

# Plotting the partial autocorrelation of New_Deaths
plt.figure(figsize=(10, 4))
plot_pacf(data['New_Deaths'], lags=50, method='ywm')
plt.title('Partial Autocorrelation of New Deaths')
plt.xlabel('Lags')
plt.ylabel('Partial Autocorrelation')
plt.show()


In [ ]:
# Load the dataset
import pandas as pd
import numpy as np

data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Generate lag features for New_Cases and New_Deaths
data['lag_1_cases'] = data['New_Cases'].shift(1)
data['lag_2_cases'] = data['New_Cases'].shift(2)
data['lag_7_cases'] = data['New_Cases'].shift(7)

data['lag_1_deaths'] = data['New_Deaths'].shift(1)
data['lag_2_deaths'] = data['New_Deaths'].shift(2)
data['lag_7_deaths'] = data['New_Deaths'].shift(7)

# Generate rolling statistics for New_Cases and New_Deaths
data['rolling_mean_7_cases'] = data['New_Cases'].rolling(window=7).mean()
data['rolling_std_7_cases'] = data['New_Cases'].rolling(window=7).std()

data['rolling_mean_7_deaths'] = data['New_Deaths'].rolling(window=7).mean()
data['rolling_std_7_deaths'] = data['New_Deaths'].rolling(window=7).std()

# Dropping any rows with NaN values created by lag/rolling features
data.dropna(inplace=True)

# Adding additional time-based features
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day
data['DayOfWeek'] = data.index.dayofweek
data['WeekOfYear'] = data.index.isocalendar().week.astype(int)

# Display the preprocessed dataset and the newly added features
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import numpy as np

# Function to evaluate the model's performance
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, r2

# Splitting the dataset for New_Cases
X_cases = data.drop(['New_Cases', 'New_Deaths'], axis=1)
y_cases = data['New_Cases']
X_train_cases, X_test_cases, y_train_cases, y_test_cases = train_test_split(X_cases, y_cases, test_size=0.2, random_state=42)

# Splitting the dataset for New_Deaths
y_deaths = data['New_Deaths']
X_train_deaths, X_test_deaths, y_train_deaths, y_test_deaths = train_test_split(X_cases, y_deaths, test_size=0.2, random_state=42)

# Define models
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=5, learning_rate=0.2, random_state=42)
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=1000, random_state=42)

# Train the models on New Cases
models = [xgb_model, rf_model, gb_model]
predictions_cases = []

for model in models:
    model.fit(X_train_cases, y_train_cases)
    preds = model.predict(X_test_cases)
    predictions_cases.append(preds)

# Averaging predictions for New Cases
final_predictions_cases = np.mean(predictions_cases, axis=0)

# Evaluate for New Cases
mse_cases, rmse_cases, mae_cases, r2_cases = evaluate_model(y_test_cases, final_predictions_cases)

# Train the models on New Deaths
predictions_deaths = []

for model in models:
    model.fit(X_train_deaths, y_train_deaths)
    preds = model.predict(X_test_deaths)
    predictions_deaths.append(preds)

# Averaging predictions for New Deaths
final_predictions_deaths = np.mean(predictions_deaths, axis=0)

# Evaluate for New Deaths
mse_deaths, rmse_deaths, mae_deaths, r2_deaths = evaluate_model(y_test_deaths, final_predictions_deaths)

# Print the evaluation metrics for New Cases
print(f"Evaluation Metrics for New Cases:")
print(f"RMSE: {rmse_cases}")
print(f"MSE: {mse_cases}")
print(f"MAE: {mae_cases}")
print(f"R-squared: {r2_cases}\n")

# Print the evaluation metrics for New Deaths
print(f"Evaluation Metrics for New Deaths:")
print(f"RMSE: {rmse_deaths}")
print(f"MSE: {mse_deaths}")
print(f"MAE: {mae_deaths}")
print(f"R-squared: {r2_deaths}")

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime format and handle any potential errors
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Drop any rows where the 'Date' conversion failed
data = data.dropna(subset=['Date'])

# Strip any extra spaces from column names
data.columns = data.columns.str.strip()

# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

# Define the last known values for cases and deaths
last_known_cases = data.iloc[-1]['New_Cases']
last_known_deaths = data.iloc[-1]['New_Deaths']
last_total_cases = data.iloc[-1]['Total_Cases']
last_total_deaths = data.iloc[-1]['Total_Deaths']

# Generate a date range for the next 365 days starting from the day after the last date in the dataset
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=365, freq='D')

# Create a DataFrame for these dates with columns ordered as per the trained model
future_data = pd.DataFrame({
    'Total_Cases': last_total_cases,
    'Total_Deaths': last_total_deaths,
    'lag_1_cases': last_known_cases,
    'lag_2_cases': last_known_cases,  # Using last known as lag_2 since we don't have it for future dates
    'lag_7_cases': last_known_cases,  # Same logic for lag_7
    'lag_1_deaths': last_known_deaths,
    'lag_2_deaths': last_known_deaths,
    'lag_7_deaths': last_known_deaths,
    'Year': future_dates.year,
    'Month': future_dates.month,
    'Day': future_dates.day,
    'DayOfWeek': future_dates.dayofweek.astype(int),
    'WeekOfYear': future_dates.isocalendar().week.astype(int)
}, index=future_dates)

# Add rolling mean and std for future dates (can be assumed constant or similar to the last known values)
future_data['rolling_mean_7_cases'] = data['New_Cases'].rolling(7).mean().iloc[-1]
future_data['rolling_std_7_cases'] = data['New_Cases'].rolling(7).std().iloc[-1]

future_data['rolling_mean_7_deaths'] = data['New_Deaths'].rolling(7).mean().iloc[-1]
future_data['rolling_std_7_deaths'] = data['New_Deaths'].rolling(7).std().iloc[-1]

# Display the first few rows of future_data
print(future_data.head())

In [ ]:
# Load and display the predictions
predicted_cases = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

print(predicted_cases.head())

In [ ]:
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skopt import BayesSearchCV

# Suppress specific LightGBM warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Custom logger to suppress LightGBM output
class SilentLogger:
    def info(self, msg):
        pass

    def warning(self, msg):
        pass

import lightgbm as lgb
lgb.register_logger(SilentLogger())

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Generate lag features for New_Cases and New_Deaths
for lag in [1, 2, 7, 14, 30]:
    data[f'lag_{lag}_cases'] = data['New_Cases'].shift(lag)
    data[f'lag_{lag}_deaths'] = data['New_Deaths'].shift(lag)

# Generate rolling statistics for New_Cases and New_Deaths
for window in [7, 14, 30]:
    data[f'rolling_mean_{window}_cases'] = data['New_Cases'].rolling(window=window).mean()
    data[f'rolling_std_{window}_cases'] = data['New_Cases'].rolling(window=window).std()
    data[f'rolling_mean_{window}_deaths'] = data['New_Deaths'].rolling(window=window).mean()
    data[f'rolling_std_{window}_deaths'] = data['New_Deaths'].rolling(window=window).std()

# Add interaction features
data['cases_deaths_interaction'] = data['New_Cases'] * data['New_Deaths']

# Dropping any rows with NaN values created by lag/rolling features
data.dropna(inplace=True)

# Adding additional time-based features
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day

# Log transform the target variables
data['log_New_Cases'] = np.log1p(data['New_Cases'])
data['log_New_Deaths'] = np.log1p(data['New_Deaths'])

# Standardize the features
features = data.drop(['New_Cases', 'New_Deaths', 'log_New_Cases', 'log_New_Deaths'], axis=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
data[features.columns] = scaled_features

# Feature importance via XGBoost to keep the most relevant features
xgb_temp = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_temp.fit(scaled_features, data['log_New_Cases'])
importance = xgb_temp.feature_importances_
important_features = features.columns[np.argsort(importance)[-20:]]  # Select top 20 features

# Reduce dimensionality using PCA
pca = PCA(n_components=10)  # Reducing dimensions to top 10 principal components
X_pca = pca.fit_transform(data[important_features])

# Update the dataset with PCA features
data_pca = pd.DataFrame(X_pca, index=data.index, columns=[f'PC{i}' for i in range(1, 11)])

# Splitting the dataset for New_Cases
X_cases = data_pca
y_cases = data['log_New_Cases']
X_train_cases, X_test_cases, y_train_cases, y_test_cases = train_test_split(X_cases, y_cases, test_size=0.2, random_state=42)

# Splitting the dataset for New_Deaths
y_deaths = data['log_New_Deaths']
X_train_deaths, X_test_deaths, y_train_deaths, y_test_deaths = train_test_split(X_cases, y_deaths, test_size=0.2, random_state=42)

# Define base models including LightGBM
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)
lgbm_model = LGBMRegressor(random_state=42)

# Fit all the models to the training data
xgb_model.fit(X_train_cases, y_train_cases)
rf_model.fit(X_train_cases, y_train_cases)
gb_model.fit(X_train_cases, y_train_cases)
lgbm_model.fit(X_train_cases, y_train_cases)

# Hyperparameter tuning for LightGBM using Bayesian Optimization
param_grid_lgbm = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'num_leaves': [31, 64, 128]
}

bayes_search_lgbm = BayesSearchCV(estimator=lgbm_model, search_spaces=param_grid_lgbm, cv=20, n_iter=100, scoring='neg_mean_squared_error', random_state=42, verbose=1)
bayes_search_lgbm.fit(X_train_cases, y_train_cases)

# Best LightGBM model
best_lgbm_model = bayes_search_lgbm.best_estimator_

# Stacking Regressor with the best models
estimators = [
    ('xgb', xgb_model),
    ('rf', rf_model),
    ('gb', gb_model),
    ('lgbm', best_lgbm_model)
]
stacking_regressor_cases = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor(random_state=42))
stacking_regressor_cases.fit(X_train_cases, y_train_cases)

stacking_regressor_deaths = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor(random_state=42))
stacking_regressor_deaths.fit(X_train_deaths, y_train_deaths)

# Function to evaluate and print model performance
def evaluate_models(models, X_test, y_test, y_test_orig, target_name):
    metrics = {}
    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_pred = np.expm1(y_pred)  # Inverse of log transformation
        mse = mean_squared_error(y_test_orig, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test_orig, y_pred)
        r2 = r2_score(y_test_orig, y_pred)
        metrics[name] = {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R-squared': r2}
        print(f"Target: {target_name} | Model: {name}")
        print(f"RMSE: {rmse}")
        print(f"MSE: {mse}")
        print(f"MAE: {mae}")
        print(f"R-squared: {r2}\n")
    return metrics

# Evaluate for New Cases
models_cases = {
    'XGBoost': xgb_model,
    'RandomForest': rf_model,
    'GradientBoosting': gb_model,
    'LightGBM': best_lgbm_model,
    'Stacking': stacking_regressor_cases
}
metrics_cases = evaluate_models(models_cases, X_test_cases, y_test_cases, np.expm1(y_test_cases), "New Cases")

# Evaluate for New Deaths
models_deaths = {
    'XGBoost': xgb_model,
    'RandomForest': rf_model,
    'GradientBoosting': gb_model,
    'LightGBM': best_lgbm_model,
    'Stacking': stacking_regressor_deaths
}
metrics_deaths = evaluate_models(models_deaths, X_test_deaths, y_test_deaths, np.expm1(y_test_deaths), "New Deaths")

In [ ]:
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA
from skopt import BayesSearchCV
from tpot import TPOTRegressor

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# TensorFlow warning suppression
tf.get_logger().setLevel('ERROR')

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# ----------------- Feature Engineering -----------------
# Generate lag features for New_Cases and New_Deaths
for lag in [1, 2, 7, 14, 30]:
    data[f'lag_{lag}_cases'] = data['New_Cases'].shift(lag)
    data[f'lag_{lag}_deaths'] = data['New_Deaths'].shift(lag)

# Generate rolling statistics for trends and moving averages
for window in [7, 14, 30]:
    data[f'rolling_mean_{window}_cases'] = data['New_Cases'].rolling(window=window).mean()
    data[f'rolling_std_{window}_cases'] = data['New_Cases'].rolling(window=window).std()
    data[f'rolling_mean_{window}_deaths'] = data['New_Deaths'].rolling(window=window).mean()
    data[f'rolling_std_{window}_deaths'] = data['New_Deaths'].rolling(window=window).std()

# Add interaction features
data['cases_deaths_interaction'] = data['New_Cases'] * data['New_Deaths']

# Adding seasonal indicator features (e.g., high season months)
data['Season'] = data.index.month.isin([6, 7, 8, 9, 10]).astype(int)

# Dropping any rows with NaN values created by lag/rolling features
data.dropna(inplace=True)

# Adding additional time-based features
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day

# Log transform the target variables
data['log_New_Cases'] = np.log1p(data['New_Cases'])
data['log_New_Deaths'] = np.log1p(data['New_Deaths'])

# Standardize the features
features = data.drop(['New_Cases', 'New_Deaths', 'log_New_Cases', 'log_New_Deaths'], axis=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
data[features.columns] = scaled_features

# Feature importance via XGBoost to keep the most relevant features
xgb_temp = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_temp.fit(scaled_features, data['log_New_Cases'])
importance = xgb_temp.feature_importances_
important_features = features.columns[np.argsort(importance)[-20:]]  # Select top 20 features

# Reduce dimensionality using PCA
pca = PCA(n_components=10)  # Reducing dimensions to top 10 principal components
X_pca = pca.fit_transform(data[important_features])

# ----------------- LSTM Model for Time Series -----------------
scaler_lstm = MinMaxScaler(feature_range=(0, 1))

def prepare_lstm_data(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Train LSTM model
def train_lstm_model(data, n_steps=60, epochs=100, batch_size=32):
    data_scaled = scaler_lstm.fit_transform(data.reshape(-1, 1))
    X, y = prepare_lstm_data(data_scaled, n_steps)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)
    return model, scaler_lstm

# Prepare data for LSTM
new_cases_data = data['New_Cases'].values
new_deaths_data = data['New_Deaths'].values

# Train LSTM models for both target variables
lstm_model_cases, scaler_cases = train_lstm_model(new_cases_data)
lstm_model_deaths, scaler_deaths = train_lstm_model(new_deaths_data)

# Function to predict with LSTM
def predict_lstm(model, data, scaler, n_steps=60):
    data_scaled = scaler.transform(data.reshape(-1, 1))
    X_test = []
    for i in range(n_steps, len(data_scaled)):
        X_test.append(data_scaled[i-n_steps:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predictions = model.predict(X_test)
    return scaler.inverse_transform(predictions)

# Make LSTM predictions
lstm_predictions_cases = predict_lstm(lstm_model_cases, new_cases_data, scaler_cases)
lstm_predictions_deaths = predict_lstm(lstm_model_deaths, new_deaths_data, scaler_deaths)

# ----------------- AutoML with TPOT -----------------
X_cases = data_pca
y_cases = data['log_New_Cases']
X_train_cases, X_test_cases, y_train_cases, y_test_cases = train_test_split(X_cases, y_cases, test_size=0.2, random_state=42)

# Use TPOT to automatically optimize and select the best model
tpot = TPOTRegressor(verbosity=2, generations=10, population_size=20, random_state=42)
tpot.fit(X_train_cases, y_train_cases)
tpot_predictions = np.expm1(tpot.predict(X_test_cases))

# Evaluate TPOT model predictions
mse_tpot = mean_squared_error(np.expm1(y_test_cases), tpot_predictions)
rmse_tpot = np.sqrt(mse_tpot)
mae_tpot = mean_absolute_error(np.expm1(y_test_cases), tpot_predictions)
r2_tpot = r2_score(np.expm1(y_test_cases), tpot_predictions)

print(f"TPOT Model - New Cases:")
print(f"RMSE: {rmse_tpot}")
print(f"MSE: {mse_tpot}")
print(f"MAE: {mae_tpot}")
print(f"R-squared: {r2_tpot}")

# ----------------- Model Evaluation -----------------
# Evaluate the LSTM model predictions for New Cases
mse_cases_lstm = mean_squared_error(new_cases_data[-len(lstm_predictions_cases):], lstm_predictions_cases)
rmse_cases_lstm = np.sqrt(mse_cases_lstm)
mae_cases_lstm = mean_absolute_error(new_cases_data[-len(lstm_predictions_cases):], lstm_predictions_cases)
r2_cases_lstm = r2_score(new_cases_data[-len(lstm_predictions_cases):], lstm_predictions_cases)

print(f"LSTM Model - New Cases:")
print(f"RMSE: {rmse_cases_lstm}")
print(f"MSE: {mse_cases_lstm}")
print(f"MAE: {mae_cases_lstm}")
print(f"R-squared: {r2_cases_lstm}")

# Evaluate the LSTM model predictions for New Deaths
mse_deaths_lstm = mean_squared_error(new_deaths_data[-len(lstm_predictions_deaths):], lstm_predictions_deaths)
rmse_deaths_lstm = np.sqrt(mse_deaths_lstm)
mae_deaths_lstm = mean_absolute_error(new_deaths_data[-len(lstm_predictions_deaths):], lstm_predictions_deaths)
r2_deaths_lstm = r2_score(new_deaths_data[-len(lstm_predictions_deaths):], lstm_predictions_deaths)

print(f"LSTM Model - New Deaths:")
print(f"RMSE: {rmse_deaths_lstm}")
print(f"MSE: {mse_deaths_lstm}")
print(f"MAE: {mae_deaths_lstm}")
print(f"R-squared: {r2_deaths_lstm}")

**end here**

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [3, 5],
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [1, 1.5]
}

# Initialize the model
xgb_model = XGBRegressor(objective='reg:squarederror')

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train_cases, y_train_cases)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best RMSE Score: {np.sqrt(best_score)}")

**RANDOM REGRESSOR**

In [ ]:
# Load the dataset
import pandas as pd

data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

print(data.head())

In [ ]:
import pandas as pd

# Load the dataset
dengue_data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime format
dengue_data['Date'] = pd.to_datetime(dengue_data['Date'], format='%m/%d/%Y')

# Set the 'Date' column as the index
dengue_data.set_index('Date', inplace=True)

# Extract month and day of the week from the index
dengue_data['Month'] = dengue_data.index.month
dengue_data['DayOfWeek'] = dengue_data.index.dayofweek

# Prepare the features for modeling
features = dengue_data[['Month', 'DayOfWeek']]  # Including the new features
target = dengue_data['Total_Deaths']  # Corrected column name


# Print the first few rows to confirm the changes
print(features.head())
print(target.head())

In [ ]:
# Import necessary libraries for modeling and validation
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import numpy as np

# Initialize the model with updated parameters
rf_model = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=42)

# Cross-validation using time series split
tscv = TimeSeriesSplit(n_splits=100)
cv_scores = cross_val_score(rf_model, features, target, cv=tscv, scoring='neg_mean_squared_error')

# Calculate RMSE for each fold
cv_rmse_scores = np.sqrt(-cv_scores)
print('Cross-validated RMSE scores:', cv_rmse_scores)
print('Average RMSE:', np.mean(cv_rmse_scores))

In [ ]:
# Split the data into training and testing sets for New Cases and New Deaths
train_cases = data['New_Cases'][:-30]
test_cases = data['New_Cases'][-30:]

train_deaths = data['New_Deaths'][:-30]
test_deaths = data['New_Deaths'][-30:]
            

In [ ]:
# Load the dataset
import pandas as pd

data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

print(data.head())
            

**time SERIS**

**NEW START AGAIN**

In [ ]:
!pip install pmdarima

In [ ]:
# Load the dataset
import pandas as pd

data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

print(data.head())
            

In [ ]:
##just ei part###

import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Apply log transformation to avoid negative predictions
data['Log_New_Cases'] = np.log1p(data['New_Cases'])  # log1p handles zeros safely
data['Log_New_Deaths'] = np.log1p(data['New_Deaths'])

# Prepare data for Prophet (New Cases)
df_new_cases = data[['Date', 'Log_New_Cases']].rename(columns={'Date': 'ds', 'Log_New_Cases': 'y'})

# Prepare data for Prophet (New Deaths)
df_new_deaths = data[['Date', 'Log_New_Deaths']].rename(columns={'Date': 'ds', 'Log_New_Deaths': 'y'})

# Initialize Prophet model for New Cases with fine-tuned hyperparameters
model_cases = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=2.0
)
model_cases.fit(df_new_cases)

# Initialize Prophet model for New Deaths with fine-tuned hyperparameters
model_deaths = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=2.0
)
model_deaths.fit(df_new_deaths)

# Generate future dates for prediction
future_cases = model_cases.make_future_dataframe(periods=60)
future_deaths = model_deaths.make_future_dataframe(periods=60)

# Forecasting
forecast_cases = model_cases.predict(future_cases)
forecast_deaths = model_deaths.predict(future_deaths)

# Inverse log transformation to bring predictions back to original scale
forecast_cases['yhat'] = np.expm1(forecast_cases['yhat'])  # expm1 reverses log1p
forecast_deaths['yhat'] = np.expm1(forecast_deaths['yhat'])

# Ensure no negative predictions by clipping at 0
forecast_cases['yhat'] = forecast_cases['yhat'].clip(lower=0)
forecast_deaths['yhat'] = forecast_deaths['yhat'].clip(lower=0)

# Merge actual and predicted values for evaluation (New Cases)
df_merged_cases = pd.merge(data[['Date', 'New_Cases']], forecast_cases[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_cases.dropna(inplace=True)

# Merge actual and predicted values for evaluation (New Deaths)
df_merged_deaths = pd.merge(data[['Date', 'New_Deaths']], forecast_deaths[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_deaths.dropna(inplace=True)

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat']))
mse_cases = mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
mae_cases = mean_absolute_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
r2_cases = r2_score(df_merged_cases['New_Cases'], df_merged_cases['yhat'])

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat']))
mse_deaths = mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
mae_deaths = mean_absolute_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
r2_deaths = r2_score(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])

# Print evaluation metrics
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(df_merged_cases['Date'], df_merged_cases['New_Cases'], label='Actual New Cases')
plt.plot(df_merged_cases['Date'], df_merged_cases['yhat'], label='Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (Scaled)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(df_merged_deaths['Date'], df_merged_deaths['New_Deaths'], label='Actual New Deaths')
plt.plot(df_merged_deaths['Date'], df_merged_deaths['yhat'], label='Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (Scaled)')
plt.legend()
plt.show()

In [ ]:
# Install required libraries
!pip install prophet scikit-learn matplotlib

import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Apply log transformation to stabilize variance
data['Log_New_Cases'] = np.log1p(data['New_Cases'])
data['Log_New_Deaths'] = np.log1p(data['New_Deaths'])

# Prepare data for Prophet (New Cases)
df_new_cases = data[['Date', 'Log_New_Cases']].rename(columns={'Date': 'ds', 'Log_New_Cases': 'y'})

# Prepare data for Prophet (New Deaths)
df_new_deaths = data[['Date', 'Log_New_Deaths']].rename(columns={'Date': 'ds', 'Log_New_Deaths': 'y'})

# Initialize Prophet model for New Cases
model_cases = Prophet(
    seasonality_mode='multiplicative', 
    yearly_seasonality=True, 
    weekly_seasonality=True,
    changepoint_prior_scale=0.1  # Controls flexibility, lower values less flexible
)
model_cases.fit(df_new_cases)

# Initialize Prophet model for New Deaths
model_deaths = Prophet(
    seasonality_mode='multiplicative', 
    yearly_seasonality=True, 
    weekly_seasonality=True,
    changepoint_prior_scale=0.1
)
model_deaths.fit(df_new_deaths)

# Generate future dates for prediction
future_cases = model_cases.make_future_dataframe(periods=60)
future_deaths = model_deaths.make_future_dataframe(periods=60)

# Forecasting
forecast_cases = model_cases.predict(future_cases)
forecast_deaths = model_deaths.predict(future_deaths)

# Inverse log transformation to bring the predictions back to the original scale
forecast_cases['yhat'] = np.expm1(forecast_cases['yhat'])
forecast_deaths['yhat'] = np.expm1(forecast_deaths['yhat'])

# Merge actual and predicted values for evaluation (New Cases)
df_merged_cases = pd.merge(data[['Date', 'New_Cases']], forecast_cases[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_cases.dropna(inplace=True)

# Merge actual and predicted values for evaluation (New Deaths)
df_merged_deaths = pd.merge(data[['Date', 'New_Deaths']], forecast_deaths[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_deaths.dropna(inplace=True)

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat']))
mse_cases = mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
mae_cases = mean_absolute_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
r2_cases = r2_score(df_merged_cases['New_Cases'], df_merged_cases['yhat'])

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat']))
mse_deaths = mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
mae_deaths = mean_absolute_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
r2_deaths = r2_score(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])

# Print evaluation metrics
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(df_merged_cases['Date'], df_merged_cases['New_Cases'], label='Actual New Cases')
plt.plot(df_merged_cases['Date'], df_merged_cases['yhat'], label='Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (Log Transformed)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(df_merged_deaths['Date'], df_merged_deaths['New_Deaths'], label='Actual New Deaths')
plt.plot(df_merged_deaths['Date'], df_merged_deaths['yhat'], label='Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (Log Transformed)')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Scaling the New Cases and New Deaths using StandardScaler
scaler_cases = StandardScaler()
scaler_deaths = StandardScaler()

# Add scaled values
data['Scaled_New_Cases'] = scaler_cases.fit_transform(data[['New_Cases']])
data['Scaled_New_Deaths'] = scaler_deaths.fit_transform(data[['New_Deaths']])

# Add a floor to prevent negative predictions
data['floor'] = 0

# Define custom seasonality (e.g., bi-weekly)
model_cases = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.1,  # Increased to better capture trend changes
    seasonality_prior_scale=1.0,  # Lowered to reduce overfitting
    holidays=pd.DataFrame({
        'holiday': 'health_intervention',
        'ds': pd.to_datetime(['2022-06-15', '2022-09-01']),  # Example dates
        'lower_window': 0,
        'upper_window': 5,
    })
).add_seasonality(name='biweekly', period=14, fourier_order=8)  # Increased Fourier order for better seasonality capture

# Similarly define model for New Deaths
model_deaths = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.1,  # Increased to better capture trend changes
    seasonality_prior_scale=1.0,  # Lowered to reduce overfitting
    holidays=pd.DataFrame({
        'holiday': 'health_intervention',
        'ds': pd.to_datetime(['2022-06-15', '2022-09-01']),  # Example dates
        'lower_window': 0,
        'upper_window': 5,
    })
)

# Fit the models
model_cases.fit(data[['Date', 'Scaled_New_Cases', 'floor']].rename(columns={'Date': 'ds', 'Scaled_New_Cases': 'y'}))
model_deaths.fit(data[['Date', 'Scaled_New_Deaths', 'floor']].rename(columns={'Date': 'ds', 'Scaled_New_Deaths': 'y'}))

# Generate future dates for prediction
future_cases = model_cases.make_future_dataframe(periods=60)
future_cases['floor'] = 0  # Add the floor constraint to future data
future_deaths = model_deaths.make_future_dataframe(periods=60)
future_deaths['floor'] = 0

# Forecasting
forecast_cases = model_cases.predict(future_cases)
forecast_deaths = model_deaths.predict(future_deaths)

# Inverse scaling to bring predictions back to original scale
forecast_cases['yhat'] = scaler_cases.inverse_transform(forecast_cases[['yhat']])
forecast_deaths['yhat'] = scaler_deaths.inverse_transform(forecast_deaths[['yhat']])

# Handle negative predictions by replacing them with zero
forecast_cases['yhat'] = np.where(forecast_cases['yhat'] < 0, 0, forecast_cases['yhat'])
forecast_deaths['yhat'] = np.where(forecast_deaths['yhat'] < 0, 0, forecast_deaths['yhat'])

# Merge actual and predicted values for evaluation (New Cases)
df_merged_cases = pd.merge(data[['Date', 'New_Cases']], forecast_cases[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_cases.dropna(inplace=True)

# Merge actual and predicted values for evaluation (New Deaths)
df_merged_deaths = pd.merge(data[['Date', 'New_Deaths']], forecast_deaths[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_deaths.dropna(inplace=True)

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat']))
mse_cases = mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
mae_cases = mean_absolute_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
r2_cases = r2_score(df_merged_cases['New_Cases'], df_merged_cases['yhat'])

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat']))
mse_deaths = mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
mae_deaths = mean_absolute_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
r2_deaths = r2_score(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])

# Print evaluation metrics
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(df_merged_cases['Date'], df_merged_cases['New_Cases'], label='Actual New Cases')
plt.plot(df_merged_cases['Date'], df_merged_cases['yhat'], label='Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (with Custom Seasonalities & Holidays)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(df_merged_deaths['Date'], df_merged_deaths['New_Deaths'], label='Actual New Deaths')
plt.plot(df_merged_deaths['Date'], df_merged_deaths['yhat'], label='Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (with Custom Seasonalities & Holidays)')
plt.legend()
plt.show()

# Optional: Perform cross-validation to ensure robustness of the model
from prophet.diagnostics import cross_validation, performance_metrics

df_cv_cases = cross_validation(model_cases, initial='365 days', period='180 days', horizon='60 days')
df_p_cases = performance_metrics(df_cv_cases)
print(df_p_cases.head())

df_cv_deaths = cross_validation(model_deaths, initial='365 days', period='180 days', horizon='60 days')
df_p_deaths = performance_metrics(df_cv_deaths)
print(df_p_deaths.head())

In [ ]:
# Install required libraries
!pip install prophet scikit-learn matplotlib

import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Remove outliers: apply thresholding to remove extreme outliers (for both New_Cases and New_Deaths)
q_low_cases = data['New_Cases'].quantile(0.01)
q_high_cases = data['New_Cases'].quantile(0.99)
q_low_deaths = data['New_Deaths'].quantile(0.01)
q_high_deaths = data['New_Deaths'].quantile(0.99)

data = data[(data['New_Cases'] > q_low_cases) & (data['New_Cases'] < q_high_cases)]
data = data[(data['New_Deaths'] > q_low_deaths) & (data['New_Deaths'] < q_high_deaths)]

# Scaling the New Cases and New Deaths using StandardScaler
scaler_cases = StandardScaler()
scaler_deaths = StandardScaler()

# Add scaled values
data['Scaled_New_Cases'] = scaler_cases.fit_transform(data[['New_Cases']])
data['Scaled_New_Deaths'] = scaler_deaths.fit_transform(data[['New_Deaths']])

# Initialize Prophet model with higher flexibility (lower changepoint_prior_scale) and adjusted fourier_order for seasonality
model_cases = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,  # Lower value for more flexibility
    seasonality_prior_scale=10.0
)

# Fine-tuning seasonality Fourier order
model_cases.add_seasonality(name='biweekly', period=14, fourier_order=7)

# Adding custom holidays for any impactful event dates (e.g., health interventions)
holidays = pd.DataFrame({
  'holiday': 'health_intervention',
  'ds': pd.to_datetime(['2022-06-15', '2022-09-01']),  # Example dates
  'lower_window': 0,
  'upper_window': 5,
})

model_cases = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=10.0,
    holidays=holidays
)

model_deaths = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=10.0,
    holidays=holidays
)

# Fit the models
model_cases.fit(data[['Date', 'Scaled_New_Cases']].rename(columns={'Date': 'ds', 'Scaled_New_Cases': 'y'}))
model_deaths.fit(data[['Date', 'Scaled_New_Deaths']].rename(columns={'Date': 'ds', 'Scaled_New_Deaths': 'y'}))

# Generate future dates for prediction
future_cases = model_cases.make_future_dataframe(periods=60)
future_deaths = model_deaths.make_future_dataframe(periods=60)

# Forecasting
forecast_cases = model_cases.predict(future_cases)
forecast_deaths = model_deaths.predict(future_deaths)

# Inverse scaling to bring predictions back to original scale
forecast_cases['yhat'] = scaler_cases.inverse_transform(forecast_cases[['yhat']])
forecast_deaths['yhat'] = scaler_deaths.inverse_transform(forecast_deaths[['yhat']])

# Merge actual and predicted values for evaluation (New Cases)
df_merged_cases = pd.merge(data[['Date', 'New_Cases']], forecast_cases[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_cases.dropna(inplace=True)

# Merge actual and predicted values for evaluation (New Deaths)
df_merged_deaths = pd.merge(data[['Date', 'New_Deaths']], forecast_deaths[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_deaths.dropna(inplace=True)

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat']))
mse_cases = mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
mae_cases = mean_absolute_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
r2_cases = r2_score(df_merged_cases['New_Cases'], df_merged_cases['yhat'])

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat']))
mse_deaths = mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
mae_deaths = mean_absolute_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
r2_deaths = r2_score(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])

# Print evaluation metrics
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(df_merged_cases['Date'], df_merged_cases['New_Cases'], label='Actual New Cases')
plt.plot(df_merged_cases['Date'], df_merged_cases['yhat'], label='Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (After Fine-Tuning)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(df_merged_deaths['Date'], df_merged_deaths['New_Deaths'], label='Actual New Deaths')
plt.plot(df_merged_deaths['Date'], df_merged_deaths['yhat'], label='Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (After Fine-Tuning)')
plt.legend()
plt.show()


In [ ]:
# Install required libraries
!pip install prophet scikit-learn matplotlib

import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Remove outliers: Apply thresholding to remove extreme outliers
q_low_cases = data['New_Cases'].quantile(0.01)
q_high_cases = data['New_Cases'].quantile(0.99)
q_low_deaths = data['New_Deaths'].quantile(0.01)
q_high_deaths = data['New_Deaths'].quantile(0.99)

data = data[(data['New_Cases'] > q_low_cases) & (data['New_Cases'] < q_high_cases)]
data = data[(data['New_Deaths'] > q_low_deaths) & (data['New_Deaths'] < q_high_deaths)]

# Add autoregressive features: Previous day’s values as a regressor
data['lag_1_cases'] = data['New_Cases'].shift(1).fillna(0)
data['lag_1_deaths'] = data['New_Deaths'].shift(1).fillna(0)

# Scaling the New Cases and New Deaths using StandardScaler
scaler_cases = StandardScaler()
scaler_deaths = StandardScaler()

data['Scaled_New_Cases'] = scaler_cases.fit_transform(data[['New_Cases']])
data['Scaled_New_Deaths'] = scaler_deaths.fit_transform(data[['New_Deaths']])

# Initialize Prophet models with autoregressive feature as a regressor
model_cases = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=10.0
)
model_cases.add_regressor('lag_1_cases')

model_deaths = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=10.0
)
model_deaths.add_regressor('lag_1_deaths')

# Fit the models
model_cases.fit(data[['Date', 'Scaled_New_Cases', 'lag_1_cases']].rename(columns={'Date': 'ds', 'Scaled_New_Cases': 'y'}))
model_deaths.fit(data[['Date', 'Scaled_New_Deaths', 'lag_1_deaths']].rename(columns={'Date': 'ds', 'Scaled_New_Deaths': 'y'}))

# Generate future dates and use last lag value for predictions
future_cases = model_cases.make_future_dataframe(periods=60)
future_cases['lag_1_cases'] = data['lag_1_cases'].iloc[-1]

future_deaths = model_deaths.make_future_dataframe(periods=60)
future_deaths['lag_1_deaths'] = data['lag_1_deaths'].iloc[-1]

# Forecasting
forecast_cases = model_cases.predict(future_cases)
forecast_deaths = model_deaths.predict(future_deaths)

# Inverse scaling and clip to avoid negative predictions
forecast_cases['yhat'] = scaler_cases.inverse_transform(forecast_cases[['yhat']])
forecast_deaths['yhat'] = scaler_deaths.inverse_transform(forecast_deaths[['yhat']])

forecast_cases['yhat'] = forecast_cases['yhat'].clip(lower=0)  # Clip negative values to 0
forecast_deaths['yhat'] = forecast_deaths['yhat'].clip(lower=0)

# Merge actual and predicted values for evaluation
df_merged_cases = pd.merge(data[['Date', 'New_Cases']], forecast_cases[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_cases.dropna(inplace=True)

df_merged_deaths = pd.merge(data[['Date', 'New_Deaths']], forecast_deaths[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_deaths.dropna(inplace=True)

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat']))
mae_cases = mean_absolute_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
r2_cases = r2_score(df_merged_cases['New_Cases'], df_merged_cases['yhat'])

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat']))
mae_deaths = mean_absolute_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
r2_deaths = r2_score(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])

# Print evaluation metrics
print(f"New Cases - RMSE: {rmse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(df_merged_cases['Date'], df_merged_cases['New_Cases'], label='Actual New Cases')
plt.plot(df_merged_cases['Date'], df_merged_cases['yhat'], label='Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (With AR Features)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(df_merged_deaths['Date'], df_merged_deaths['New_Deaths'], label='Actual New Deaths')
plt.plot(df_merged_deaths['Date'], df_merged_deaths['yhat'], label='Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (With AR Features)')
plt.legend()
plt.show()


In [ ]:
# Install required libraries
!pip install prophet scikit-learn matplotlib

import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Remove outliers: apply thresholding to remove extreme outliers (for both New_Cases and New_Deaths)
q_low_cases = data['New_Cases'].quantile(0.01)
q_high_cases = data['New_Cases'].quantile(0.99)
q_low_deaths = data['New_Deaths'].quantile(0.01)
q_high_deaths = data['New_Deaths'].quantile(0.99)

data = data[(data['New_Cases'] > q_low_cases) & (data['New_Cases'] < q_high_cases)]
data = data[(data['New_Deaths'] > q_low_deaths) & (data['New_Deaths'] < q_high_deaths)]

# Add autoregressive features: previous days’ values as regressors (lag_1, lag_2)
data['lag_1_cases'] = data['New_Cases'].shift(1).fillna(0)
data['lag_2_cases'] = data['New_Cases'].shift(2).fillna(0)

data['lag_1_deaths'] = data['New_Deaths'].shift(1).fillna(0)
data['lag_2_deaths'] = data['New_Deaths'].shift(2).fillna(0)

# Scaling the New Cases and New Deaths using StandardScaler
scaler_cases = StandardScaler()
scaler_deaths = StandardScaler()

# Add scaled values
data['Scaled_New_Cases'] = scaler_cases.fit_transform(data[['New_Cases']])
data['Scaled_New_Deaths'] = scaler_deaths.fit_transform(data[['New_Deaths']])

# Initialize Prophet models with autoregressive features as regressors
model_cases = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=10.0
)
model_cases.add_regressor('lag_1_cases')  # Adding autoregressive feature lag_1
model_cases.add_regressor('lag_2_cases')  # Adding autoregressive feature lag_2

model_deaths = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=10.0
)
model_deaths.add_regressor('lag_1_deaths')  # Adding autoregressive feature lag_1
model_deaths.add_regressor('lag_2_deaths')  # Adding autoregressive feature lag_2

# Fit the models
model_cases.fit(data[['Date', 'Scaled_New_Cases', 'lag_1_cases', 'lag_2_cases']].rename(columns={'Date': 'ds', 'Scaled_New_Cases': 'y'}))
model_deaths.fit(data[['Date', 'Scaled_New_Deaths', 'lag_1_deaths', 'lag_2_deaths']].rename(columns={'Date': 'ds', 'Scaled_New_Deaths': 'y'}))

# Generate future dates for prediction and add autoregressive features (lag_1_cases, lag_2_cases)
future_cases = model_cases.make_future_dataframe(periods=60)
future_cases['lag_1_cases'] = data['lag_1_cases'].iloc[-1]  # Using last value for lag_1_cases
future_cases['lag_2_cases'] = data['lag_2_cases'].iloc[-1]  # Using last value for lag_2_cases

future_deaths = model_deaths.make_future_dataframe(periods=60)
future_deaths['lag_1_deaths'] = data['lag_1_deaths'].iloc[-1]  # Using last value for lag_1_deaths
future_deaths['lag_2_deaths'] = data['lag_2_deaths'].iloc[-1]  # Using last value for lag_2_deaths

# Forecasting
forecast_cases = model_cases.predict(future_cases)
forecast_deaths = model_deaths.predict(future_deaths)

# Inverse scaling to bring predictions back to original scale
forecast_cases['yhat'] = scaler_cases.inverse_transform(forecast_cases[['yhat']])
forecast_deaths['yhat'] = scaler_deaths.inverse_transform(forecast_deaths[['yhat']])

# Merge actual and predicted values for evaluation (New Cases)
df_merged_cases = pd.merge(data[['Date', 'New_Cases']], forecast_cases[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_cases.dropna(inplace=True)

# Merge actual and predicted values for evaluation (New Deaths)
df_merged_deaths = pd.merge(data[['Date', 'New_Deaths']], forecast_deaths[['ds', 'yhat']], left_on='Date', right_on='ds', how='left')
df_merged_deaths.dropna(inplace=True)

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat']))
mse_cases = mean_squared_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
mae_cases = mean_absolute_error(df_merged_cases['New_Cases'], df_merged_cases['yhat'])
r2_cases = r2_score(df_merged_cases['New_Cases'], df_merged_cases['yhat'])

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat']))
mse_deaths = mean_squared_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
mae_deaths = mean_absolute_error(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])
r2_deaths = r2_score(df_merged_deaths['New_Deaths'], df_merged_deaths['yhat'])

# Print evaluation metrics
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(df_merged_cases['Date'], df_merged_cases['New_Cases'], label='Actual New Cases')
plt.plot(df_merged_cases['Date'], df_merged_cases['yhat'], label='Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (With Lag 1 and Lag 2 AR Features)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(df_merged_deaths['Date'], df_merged_deaths['New_Deaths'], label='Actual New Deaths')
plt.plot(df_merged_deaths['Date'], df_merged_deaths['yhat'], label='Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (With Lag 1 and Lag 2 AR Features)')
plt.legend()
plt.show()


**ENsumblw with prophet**

In [ ]:
!pip install prophet pmdarima xgboost scikit-learn matplotlib

In [ ]:
import pandas as pd
from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Prepare data for modeling
data['lag_1_cases'] = data['New_Cases'].shift(1).fillna(0)
data['lag_2_cases'] = data['New_Cases'].shift(2).fillna(0)
data['lag_1_deaths'] = data['New_Deaths'].shift(1).fillna(0)
data['lag_2_deaths'] = data['New_Deaths'].shift(2).fillna(0)

# Drop rows with missing values
data = data.dropna()

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)


In [ ]:
# Prophet Model for New Cases
prophet_cases = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=True)
prophet_cases.fit(train_data[['Date', 'New_Cases']].rename(columns={'Date': 'ds', 'New_Cases': 'y'}))

# Prophet Model for New Deaths
prophet_deaths = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=True)
prophet_deaths.fit(train_data[['Date', 'New_Deaths']].rename(columns={'Date': 'ds', 'New_Deaths': 'y'}))

# Make future predictions for Prophet
future_dates = prophet_cases.make_future_dataframe(periods=len(test_data), freq='D')
forecast_cases = prophet_cases.predict(future_dates)
forecast_deaths = prophet_deaths.predict(future_dates)

# Get the predicted values from Prophet
prophet_cases_preds = forecast_cases[['ds', 'yhat']].tail(len(test_data))['yhat'].values
prophet_deaths_preds = forecast_deaths[['ds', 'yhat']].tail(len(test_data))['yhat'].values

In [ ]:
# SARIMA for New Cases
sarima_cases = auto_arima(train_data['New_Cases'], seasonal=True, stepwise=True)
sarima_cases_preds = sarima_cases.predict(n_periods=len(test_data))

# SARIMA for New Deaths
sarima_deaths = auto_arima(train_data['New_Deaths'], seasonal=True, stepwise=True)
sarima_deaths_preds = sarima_deaths.predict(n_periods=len(test_data))

In [ ]:
# XGBoost for New Cases
xgb_cases = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_cases.fit(train_data[['lag_1_cases', 'lag_2_cases']], train_data['New_Cases'])
xgb_cases_preds = xgb_cases.predict(test_data[['lag_1_cases', 'lag_2_cases']])

# XGBoost for New Deaths
xgb_deaths = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_deaths.fit(train_data[['lag_1_deaths', 'lag_2_deaths']], train_data['New_Deaths'])
xgb_deaths_preds = xgb_deaths.predict(test_data[['lag_1_deaths', 'lag_2_deaths']])

In [ ]:
# Combining predictions for New Cases
final_cases_preds = 0.4 * prophet_cases_preds + 0.3 * sarima_cases_preds + 0.3 * xgb_cases_preds

# Combining predictions for New Deaths
final_deaths_preds = 0.4 * prophet_deaths_preds + 0.3 * sarima_deaths_preds + 0.3 * xgb_deaths_preds

# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(test_data['New_Cases'], final_cases_preds))
mse_cases = mean_squared_error(test_data['New_Cases'], final_cases_preds)
mae_cases = mean_absolute_error(test_data['New_Cases'], final_cases_preds)
r2_cases = r2_score(test_data['New_Cases'], final_cases_preds)

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(test_data['New_Deaths'], final_deaths_preds))
mse_deaths = mean_squared_error(test_data['New_Deaths'], final_deaths_preds)
mae_deaths = mean_absolute_error(test_data['New_Deaths'], final_deaths_preds)
r2_deaths = r2_score(test_data['New_Deaths'], final_deaths_preds)

# Print results
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")

In [ ]:
# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(test_data['Date'], test_data['New_Cases'], label='Actual New Cases')
plt.plot(test_data['Date'], final_cases_preds, label='Predicted New Cases (Ensemble)')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (Ensemble)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(test_data['Date'], test_data['New_Deaths'], label='Actual New Deaths')
plt.plot(test_data['Date'], final_deaths_preds, label='Predicted New Deaths (Ensemble)')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (Ensemble)')
plt.legend()
plt.show()

In [ ]:
from pmdarima import auto_arima

# Auto ARIMA for New Cases
sarima_cases = auto_arima(train_data['New_Cases'], seasonal=True, stepwise=True, trace=True)

# Auto ARIMA for New Deaths
sarima_deaths = auto_arima(train_data['New_Deaths'], seasonal=True, stepwise=True, trace=True)

# SARIMA Predictions
sarima_cases_preds = sarima_cases.predict(n_periods=len(test_data))
sarima_deaths_preds = sarima_deaths.predict(n_periods=len(test_data))


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Prepare the training data for XGBoost
X_train_cases = train_data[['lag_1_cases', 'lag_2_cases']]
y_train_cases = train_data['New_Cases']
X_train_deaths = train_data[['lag_1_deaths', 'lag_2_deaths']]
y_train_deaths = train_data['New_Deaths']

# Define parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Hyperparameter tuning for XGBoost on New Cases
xgb_cases = XGBRegressor()
grid_cases = GridSearchCV(xgb_cases, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_cases.fit(X_train_cases, y_train_cases)
best_xgb_cases = grid_cases.best_estimator_

# Hyperparameter tuning for XGBoost on New Deaths
xgb_deaths = XGBRegressor()
grid_deaths = GridSearchCV(xgb_deaths, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_deaths.fit(X_train_deaths, y_train_deaths)
best_xgb_deaths = grid_deaths.best_estimator_

# Make predictions
xgb_cases_preds = best_xgb_cases.predict(test_data[['lag_1_cases', 'lag_2_cases']])
xgb_deaths_preds = best_xgb_deaths.predict(test_data[['lag_1_deaths', 'lag_2_deaths']])


In [ ]:
# Prophet Model for New Cases
prophet_cases = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=True)
prophet_cases.fit(train_data[['Date', 'New_Cases']].rename(columns={'Date': 'ds', 'New_Cases': 'y'}))

# Prophet Model for New Deaths
prophet_deaths = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=True)
prophet_deaths.fit(train_data[['Date', 'New_Deaths']].rename(columns={'Date': 'ds', 'New_Deaths': 'y'}))

# Make future predictions for Prophet
future_dates = prophet_cases.make_future_dataframe(periods=len(test_data), freq='D')
forecast_cases = prophet_cases.predict(future_dates)
forecast_deaths = prophet_deaths.predict(future_dates)

# Get the predicted values from Prophet
prophet_cases_preds = forecast_cases[['ds', 'yhat']].tail(len(test_data))['yhat'].values
prophet_deaths_preds = forecast_deaths[['ds', 'yhat']].tail(len(test_data))['yhat'].values


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# First, make predictions with SARIMA and treat them as features
sarima_cases_preds = sarima_cases.predict(n_periods=len(test_data))
sarima_deaths_preds = sarima_deaths.predict(n_periods=len(test_data))

# Prepare the stacking features (SARIMA predictions are treated as features)
X_train_cases_stacking = np.column_stack([prophet_cases_preds, xgb_cases_preds, sarima_cases_preds])
X_train_deaths_stacking = np.column_stack([prophet_deaths_preds, xgb_deaths_preds, sarima_deaths_preds])

# Prepare the stacking regressor (Prophet, XGBoost are used as regressors)
stacking_cases = StackingRegressor(
    estimators=[
        ('prophet', best_xgb_cases),  # XGBoost model
        ('xgb', best_xgb_cases),  # XGBoost model
    ],
    final_estimator=Ridge()
)

stacking_deaths = StackingRegressor(
    estimators=[
        ('prophet', best_xgb_deaths),  # XGBoost model
        ('xgb', best_xgb_deaths),  # XGBoost model
    ],
    final_estimator=Ridge()
)

# Train the stacking model using SARIMA predictions as features
stacking_cases.fit(X_train_cases_stacking, test_data['New_Cases'])
stacking_deaths.fit(X_train_deaths_stacking, test_data['New_Deaths'])

# Make predictions with the stacking model
stacked_cases_preds = stacking_cases.predict(X_train_cases_stacking)
stacked_deaths_preds = stacking_deaths.predict(X_train_deaths_stacking)


In [ ]:
# Evaluation metrics for New Cases
rmse_cases = np.sqrt(mean_squared_error(test_data['New_Cases'], stacked_cases_preds))
mse_cases = mean_squared_error(test_data['New_Cases'], stacked_cases_preds)
mae_cases = mean_absolute_error(test_data['New_Cases'], stacked_cases_preds)
r2_cases = r2_score(test_data['New_Cases'], stacked_cases_preds)

# Evaluation metrics for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(test_data['New_Deaths'], stacked_deaths_preds))
mse_deaths = mean_squared_error(test_data['New_Deaths'], stacked_deaths_preds)
mae_deaths = mean_absolute_error(test_data['New_Deaths'], stacked_deaths_preds)
r2_deaths = r2_score(test_data['New_Deaths'], stacked_deaths_preds)

# Print results
print(f"New Cases - RMSE: {rmse_cases}, MSE: {mse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"New Deaths - RMSE: {rmse_deaths}, MSE: {mse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")


In [ ]:
# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(test_data['Date'], test_data['New_Cases'], label='Actual New Cases')
plt.plot(test_data['Date'], stacked_cases_preds, label='Predicted New Cases (Ensemble)')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (Stacking Ensemble)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(test_data['Date'], test_data['New_Deaths'], label='Actual New Deaths')
plt.plot(test_data['Date'], stacked_deaths_preds, label='Predicted New Deaths (Stacking Ensemble)')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (Stacking Ensemble)')
plt.legend()
plt.show()


**new ensamble**

In [ ]:
# Necessary imports
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Remove outliers based on quantiles
q_low_cases = data['New_Cases'].quantile(0.01)
q_high_cases = data['New_Cases'].quantile(0.99)
q_low_deaths = data['New_Deaths'].quantile(0.01)
q_high_deaths = data['New_Deaths'].quantile(0.99)

data = data[(data['New_Cases'] > q_low_cases) & (data['New_Cases'] < q_high_cases)]
data = data[(data['New_Deaths'] > q_low_deaths) & (data['New_Deaths'] < q_high_deaths)]

# Add lag features for short-term dependencies
data['lag_1_cases'] = data['New_Cases'].shift(1).fillna(0)
data['lag_2_cases'] = data['New_Cases'].shift(2).fillna(0)
data['lag_1_deaths'] = data['New_Deaths'].shift(1).fillna(0)
data['lag_2_deaths'] = data['New_Deaths'].shift(2).fillna(0)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

# Display first few rows of the training data
train_data.head()


In [ ]:
from prophet import Prophet

# Prophet Model for New Cases
prophet_cases = Prophet(yearly_seasonality=True, weekly_seasonality=True, changepoint_prior_scale=0.01)
prophet_cases.fit(train_data[['Date', 'New_Cases']].rename(columns={'Date': 'ds', 'New_Cases': 'y'}))

# Prophet Model for New Deaths
prophet_deaths = Prophet(yearly_seasonality=True, weekly_seasonality=True, changepoint_prior_scale=0.01)
prophet_deaths.fit(train_data[['Date', 'New_Deaths']].rename(columns={'Date': 'ds', 'New_Deaths': 'y'}))

# Predictions
future_dates = prophet_cases.make_future_dataframe(periods=len(test_data))
prophet_cases_preds = prophet_cases.predict(future_dates)['yhat'][-len(test_data):]
prophet_deaths_preds = prophet_deaths.predict(future_dates)['yhat'][-len(test_data):]

# Evaluate Prophet Results
prophet_rmse_cases = np.sqrt(mean_squared_error(test_data['New_Cases'], prophet_cases_preds))
prophet_rmse_deaths = np.sqrt(mean_squared_error(test_data['New_Deaths'], prophet_deaths_preds))
print(f"Prophet New Cases RMSE: {prophet_rmse_cases}")
print(f"Prophet New Deaths RMSE: {prophet_rmse_deaths}")

In [ ]:
from pmdarima import auto_arima

# SARIMA for New Cases
sarima_cases = auto_arima(train_data['New_Cases'], seasonal=True, stepwise=True, trace=True)
sarima_cases_preds = sarima_cases.predict(n_periods=len(test_data))

# SARIMA for New Deaths
sarima_deaths = auto_arima(train_data['New_Deaths'], seasonal=True, stepwise=True, trace=True)
sarima_deaths_preds = sarima_deaths.predict(n_periods=len(test_data))

# Evaluate SARIMA Results
sarima_rmse_cases = np.sqrt(mean_squared_error(test_data['New_Cases'], sarima_cases_preds))
sarima_rmse_deaths = np.sqrt(mean_squared_error(test_data['New_Deaths'], sarima_deaths_preds))
print(f"SARIMA New Cases RMSE: {sarima_rmse_cases}")
print(f"SARIMA New Deaths RMSE: {sarima_rmse_deaths}")

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Grid Search for XGBoost hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}
xgb_cases = XGBRegressor()
grid_cases = GridSearchCV(xgb_cases, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_cases.fit(train_data[['lag_1_cases', 'lag_2_cases']], train_data['New_Cases'])

xgb_deaths = XGBRegressor()
grid_deaths = GridSearchCV(xgb_deaths, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_deaths.fit(train_data[['lag_1_deaths', 'lag_2_deaths']], train_data['New_Deaths'])

# Predictions for XGBoost
xgb_cases_preds = grid_cases.best_estimator_.predict(test_data[['lag_1_cases', 'lag_2_cases']])
xgb_deaths_preds = grid_deaths.best_estimator_.predict(test_data[['lag_1_deaths', 'lag_2_deaths']])

# Evaluate XGBoost Results
xgb_rmse_cases = np.sqrt(mean_squared_error(test_data['New_Cases'], xgb_cases_preds))
xgb_rmse_deaths = np.sqrt(mean_squared_error(test_data['New_Deaths'], xgb_deaths_preds))
print(f"XGBoost New Cases RMSE: {xgb_rmse_cases}")
print(f"XGBoost New Deaths RMSE: {xgb_rmse_deaths}")


In [ ]:
# Weighting based on performance
weights = {
    'prophet': 0.4,  # Give Prophet more weight if it performs better in capturing trends
    'sarima': 0.3,   # SARIMA for seasonality and short-term patterns
    'xgb': 0.3       # XGBoost for short-term fluctuations
}

# Final ensemble predictions (weighted average)
final_cases_preds = (weights['prophet'] * prophet_cases_preds) + (weights['sarima'] * sarima_cases_preds) + (weights['xgb'] * xgb_cases_preds)
final_deaths_preds = (weights['prophet'] * prophet_deaths_preds) + (weights['sarima'] * sarima_deaths_preds) + (weights['xgb'] * xgb_deaths_preds)

# Evaluate Ensemble
rmse_cases_ensemble = np.sqrt(mean_squared_error(test_data['New_Cases'], final_cases_preds))
rmse_deaths_ensemble = np.sqrt(mean_squared_error(test_data['New_Deaths'], final_deaths_preds))

print(f"Ensemble New Cases RMSE: {rmse_cases_ensemble}")
print(f"Ensemble New Deaths RMSE: {rmse_deaths_ensemble}")


In [ ]:
# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(test_data['Date'], test_data['New_Cases'], label='Actual New Cases')
plt.plot(test_data['Date'], final_cases_preds, label='Ensemble Predicted New Cases')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (Ensemble)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(test_data['Date'], test_data['New_Deaths'], label='Actual New Deaths')
plt.plot(test_data['Date'], final_deaths_preds, label='Ensemble Predicted New Deaths')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (Ensemble)')
plt.legend()
plt.show()


**Deep learning way**

In [ ]:
!pip install tensorflow scikit-learn matplotlib pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Remove outliers based on quantiles
q_low_cases = data['New_Cases'].quantile(0.01)
q_high_cases = data['New_Cases'].quantile(0.99)
q_low_deaths = data['New_Deaths'].quantile(0.01)
q_high_deaths = data['New_Deaths'].quantile(0.99)

data = data[(data['New_Cases'] > q_low_cases) & (data['New_Cases'] < q_high_cases)]
data = data[(data['New_Deaths'] > q_low_deaths) & (data['New_Deaths'] < q_high_deaths)]

# Prepare features and targets for LSTM
data = data[['Date', 'New_Cases', 'New_Deaths']]
data.set_index('Date', inplace=True)

# Scale the data
scaler_cases = MinMaxScaler(feature_range=(0, 1))
scaled_cases = scaler_cases.fit_transform(data[['New_Cases']])

scaler_deaths = MinMaxScaler(feature_range=(0, 1))
scaled_deaths = scaler_deaths.fit_transform(data[['New_Deaths']])

# Convert data to time-series format for LSTM
def create_lstm_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

# Create datasets for cases and deaths
time_step = 7  # Considering 7 days of history for predictions
X_cases, Y_cases = create_lstm_dataset(scaled_cases, time_step)
X_deaths, Y_deaths = create_lstm_dataset(scaled_deaths, time_step)

# Reshape input to be [samples, time steps, features] which is required for LSTM
X_cases = X_cases.reshape(X_cases.shape[0], X_cases.shape[1], 1)
X_deaths = X_deaths.reshape(X_deaths.shape[0], X_deaths.shape[1], 1)

# Split into train and test
split_ratio = 0.8
split_idx_cases = int(len(X_cases) * split_ratio)
split_idx_deaths = int(len(X_deaths) * split_ratio)

X_train_cases, X_test_cases = X_cases[:split_idx_cases], X_cases[split_idx_cases:]
Y_train_cases, Y_test_cases = Y_cases[:split_idx_cases], Y_cases[split_idx_cases:]

X_train_deaths, X_test_deaths = X_deaths[:split_idx_deaths], X_deaths[split_idx_deaths:]
Y_train_deaths, Y_test_deaths = Y_deaths[:split_idx_deaths], Y_deaths[split_idx_deaths:]

In [ ]:
#gpt# vai er ta run hoccchilo na#
# Import necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Example dataset (replace with your actual dataset)
cases_data = np.random.rand(1000, 1)  # Replace with actual data

# Scale the data
scaler_cases = MinMaxScaler(feature_range=(0, 1))
scaled_cases_data = scaler_cases.fit_transform(cases_data)

# Define the time step
time_step = 10  # The number of time steps for LSTM input
X, Y = [], []

# Create the sequences for LSTM
for i in range(time_step, len(scaled_cases_data)):
    X.append(scaled_cases_data[i-time_step:i, 0])  # Create input sequences
    Y.append(scaled_cases_data[i, 0])  # Create output sequences (next step)

# Convert to numpy arrays
X, Y = np.array(X), np.array(Y)

# Reshape X for LSTM input (samples, time steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Reshape to (samples, time steps, 1 feature)

# Split the data into training and testing sets
X_train_cases, X_test_cases, Y_train_cases, Y_test_cases = train_test_split(X, Y, test_size=0.2, random_state=42)

# LSTM model for New Cases
model_cases = Sequential()
model_cases.add(LSTM(100, return_sequences=True, input_shape=(time_step, 1)))
model_cases.add(Dropout(0.2))
model_cases.add(LSTM(50, return_sequences=False))
model_cases.add(Dropout(0.2))
model_cases.add(Dense(1))

# Compile the model
model_cases.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model_cases.fit(X_train_cases, Y_train_cases, epochs=20, batch_size=32, verbose=1)

# Predict on test data
pred_cases = model_cases.predict(X_test_cases)

# Reshape Y_test_cases before inverse transformation
Y_test_cases = Y_test_cases.reshape(-1, 1)

# Inverse transform the predictions and actual values to original scale
pred_cases = scaler_cases.inverse_transform(pred_cases)
Y_test_cases = scaler_cases.inverse_transform(Y_test_cases)

# Calculate RMSE for New Cases
rmse_cases = np.sqrt(mean_squared_error(Y_test_cases, pred_cases))
print(f"LSTM New Cases RMSE: {rmse_cases}")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load the dataset (update the file path to your dataset)
# Ensure that the dataset has a column named 'New_Deaths'
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')

# Check if the 'New_Deaths' column exists
if 'New_Deaths' not in data.columns:
    raise ValueError("'New_Deaths' column is missing from the dataset!")

# Define the column from your data representing New Deaths
new_deaths = data['New_Deaths'].values

# Scaling the data
scaler_deaths = MinMaxScaler(feature_range=(0, 1))
new_deaths_scaled = scaler_deaths.fit_transform(new_deaths.reshape(-1, 1))

# Create a function to prepare your dataset for LSTM
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

# Define the time_step for LSTM
time_step = 10
X, Y = create_dataset(new_deaths_scaled, time_step)

# Reshape the input to be [samples, time steps, features] as required by LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the dataset into training and testing data
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train_deaths, X_test_deaths = X[0:train_size], X[train_size:len(X)]
Y_train_deaths, Y_test_deaths = Y[0:train_size], Y[train_size:len(Y)]

# Define the LSTM model
model_deaths = Sequential()
model_deaths.add(LSTM(100, return_sequences=True, input_shape=(time_step, 1)))
model_deaths.add(Dropout(0.2))
model_deaths.add(LSTM(50, return_sequences=False))
model_deaths.add(Dropout(0.2))
model_deaths.add(Dense(1))

# Compile the model
model_deaths.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model_deaths.fit(X_train_deaths, Y_train_deaths, epochs=20, batch_size=32, verbose=1)

# Predict on test data
pred_deaths = model_deaths.predict(X_test_deaths)

# Inverse transform the predictions to original scale
pred_deaths = scaler_deaths.inverse_transform(pred_deaths)
Y_test_deaths = scaler_deaths.inverse_transform([Y_test_deaths])

# Calculate RMSE for New Deaths
rmse_deaths = np.sqrt(mean_squared_error(Y_test_deaths[0], pred_deaths[:, 0]))
print(f"LSTM New Deaths RMSE: {rmse_deaths}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ============================
# Define Your Actual Data Here
# ============================

# Example: Replace the following synthetic data with your actual test data and predictions.

# Number of time steps in the test set
num_time_steps_cases = 50
num_time_steps_deaths = 50

# Synthetic actual new cases data (Replace with your actual Y_test_cases)
# Shape: (num_time_steps_cases, 1)
Y_test_cases = np.random.randint(50, 100, size=(num_time_steps_cases, 1))

# Synthetic predicted new cases data (Replace with your actual pred_cases)
# Shape: (num_time_steps_cases, 1)
pred_cases = Y_test_cases + np.random.randint(-10, 10, size=(num_time_steps_cases, 1))

# Synthetic actual new deaths data (Replace with your actual Y_test_deaths)
# Shape: (num_time_steps_deaths, 1)
Y_test_deaths = np.random.randint(5, 20, size=(num_time_steps_deaths, 1))

# Synthetic predicted new deaths data (Replace with your actual pred_deaths)
# Shape: (num_time_steps_deaths, 1)
pred_deaths = Y_test_deaths + np.random.randint(-3, 3, size=(num_time_steps_deaths, 1))

# ====================================
# Define Time Indices Based on Data
# ====================================

# If you have actual dates, you can use them instead of a range of integers.
# For example, if you have a list of dates called `dates`, you can use:
# time_indices_cases = dates[-num_time_steps_cases:]
# time_indices_deaths = dates[-num_time_steps_deaths:]

# Using a range of integers as time indices
time_indices_cases = np.arange(len(Y_test_cases))
time_indices_deaths = np.arange(len(Y_test_deaths))

# ========================
# Plotting the Predictions
# ========================

# Plot predictions for New Cases
plt.figure(figsize=(10, 6))
plt.plot(time_indices_cases, Y_test_cases[:, 0], label='Actual New Cases', marker='o')
plt.plot(time_indices_cases, pred_cases[:, 0], label='Predicted New Cases', marker='x')
plt.title('Actual vs Predicted New Cases (LSTM)')
plt.xlabel('Time')
plt.ylabel('New Cases')
plt.legend()
plt.grid(True)
plt.show()

# Plot predictions for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(time_indices_deaths, Y_test_deaths[:, 0], label='Actual New Deaths', marker='o')
plt.plot(time_indices_deaths, pred_deaths[:, 0], label='Predicted New Deaths', marker='x')
plt.title('Actual vs Predicted New Deaths (LSTM)')
plt.xlabel('Time')
plt.ylabel('New Deaths')
plt.legend()
plt.grid(True)
plt.show()


**Hybrid prpht, Xgiboost Sarima,Lstm with prophet**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Load the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data.set_index('Date', inplace=True)

# Decompose the time series for New Cases and New Deaths
result_cases = seasonal_decompose(data['New_Cases'], model='additive', period=7)
result_deaths = seasonal_decompose(data['New_Deaths'], model='additive', period=7)

# Plot the decomposition
result_cases.plot()
plt.title('Decomposition of New Cases')
plt.show()

result_deaths.plot()
plt.title('Decomposition of New Deaths')
plt.show()

In [ ]:
# Add time-based features
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['week_of_year'] = data.index.isocalendar().week

# Add rolling statistics for smoothing
data['rolling_mean_cases'] = data['New_Cases'].rolling(window=7).mean().fillna(method='bfill')
data['rolling_std_cases'] = data['New_Cases'].rolling(window=7).std().fillna(method='bfill')
data['rolling_mean_deaths'] = data['New_Deaths'].rolling(window=7).mean().fillna(method='bfill')
data['rolling_std_deaths'] = data['New_Deaths'].rolling(window=7).std().fillna(method='bfill')

# Differencing to achieve stationarity
data['diff_cases'] = data['New_Cases'].diff().fillna(0)
data['diff_deaths'] = data['New_Deaths'].diff().fillna(0)

In [ ]:
!pip install pmdarima


In [ ]:
from pmdarima import auto_arima

# SARIMA for New Cases
sarima_cases = auto_arima(data['New_Cases'], seasonal=True, stepwise=True)
sarima_cases_preds = sarima_cases.predict(n_periods=len(data))

# SARIMA for New Deaths
sarima_deaths = auto_arima(data['New_Deaths'], seasonal=True, stepwise=True)
sarima_deaths_preds = sarima_deaths.predict(n_periods=len(data))

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Scaling the data
scaler_cases = MinMaxScaler(feature_range=(0, 1))
scaled_cases = scaler_cases.fit_transform(data[['New_Cases']])

scaler_deaths = MinMaxScaler(feature_range=(0, 1))
scaled_deaths = scaler_deaths.fit_transform(data[['New_Deaths']])

# Function to create dataset for LSTM
def create_lstm_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 7  # History of 7 days
X_cases, Y_cases = create_lstm_dataset(scaled_cases, time_step)
X_deaths, Y_deaths = create_lstm_dataset(scaled_deaths, time_step)

# Reshape input for LSTM [samples, time steps, features]
X_cases = X_cases.reshape(X_cases.shape[0], X_cases.shape[1], 1)
X_deaths = X_deaths.reshape(X_deaths.shape[0], X_deaths.shape[1], 1)

# LSTM Model for New Cases
model_cases = Sequential()
model_cases.add(LSTM(100, return_sequences=True, input_shape=(time_step, 1)))
model_cases.add(Dropout(0.2))
model_cases.add(LSTM(50, return_sequences=False))
model_cases.add(Dropout(0.2))
model_cases.add(Dense(1))

model_cases.compile(optimizer='adam', loss='mean_squared_error')
model_cases.fit(X_cases, Y_cases, epochs=20, batch_size=32, verbose=1)

# LSTM Model for New Deaths
model_deaths = Sequential()
model_deaths.add(LSTM(100, return_sequences=True, input_shape=(time_step, 1)))
model_deaths.add(Dropout(0.2))
model_deaths.add(LSTM(50, return_sequences=False))
model_deaths.add(Dropout(0.2))
model_deaths.add(Dense(1))

model_deaths.compile(optimizer='adam', loss='mean_squared_error')
model_deaths.fit(X_deaths, Y_deaths, epochs=20, batch_size=32, verbose=1)

# Predictions
pred_cases = model_cases.predict(X_cases)
pred_deaths = model_deaths.predict(X_deaths)

# Inverse scaling
pred_cases = scaler_cases.inverse_transform(pred_cases)
pred_deaths = scaler_deaths.inverse_transform(pred_deaths)

In [ ]:
from xgboost import XGBRegressor

# XGBoost for New Cases
xgb_cases = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_cases.fit(data[['day_of_week', 'month', 'rolling_mean_cases', 'rolling_std_cases']], data['New_Cases'])
xgb_cases_preds = xgb_cases.predict(data[['day_of_week', 'month', 'rolling_mean_cases', 'rolling_std_cases']])

# XGBoost for New Deaths
xgb_deaths = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_deaths.fit(data[['day_of_week', 'month', 'rolling_mean_deaths', 'rolling_std_deaths']], data['New_Deaths'])
xgb_deaths_preds = xgb_deaths.predict(data[['day_of_week', 'month', 'rolling_mean_deaths', 'rolling_std_deaths']])

In [ ]:
# Find the minimum length of predictions
min_length_cases = min(len(sarima_cases_preds), len(pred_cases), len(xgb_cases_preds))
min_length_deaths = min(len(sarima_deaths_preds), len(pred_deaths), len(xgb_deaths_preds))

# Truncate predictions to the same length
sarima_cases_preds = sarima_cases_preds[:min_length_cases]
pred_cases = pred_cases[:min_length_cases]
xgb_cases_preds = xgb_cases_preds[:min_length_cases]

sarima_deaths_preds = sarima_deaths_preds[:min_length_deaths]
pred_deaths = pred_deaths[:min_length_deaths]
xgb_deaths_preds = xgb_deaths_preds[:min_length_deaths]

# Ensemble with simple averaging (after truncating)
final_cases_preds = (0.3 * sarima_cases_preds) + (0.4 * pred_cases.flatten()) + (0.3 * xgb_cases_preds)
final_deaths_preds = (0.3 * sarima_deaths_preds) + (0.4 * pred_deaths.flatten()) + (0.3 * xgb_deaths_preds)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error  # Import the missing function

# Truncate the actual values for cases and deaths
actual_cases = data['New_Cases'][-min_length_cases:]
actual_deaths = data['New_Deaths'][-min_length_deaths:]

# Evaluation metrics
rmse_cases = np.sqrt(mean_squared_error(actual_cases, final_cases_preds))
rmse_deaths = np.sqrt(mean_squared_error(actual_deaths, final_deaths_preds))

print(f"Final RMSE New Cases: {rmse_cases}")
print(f"Final RMSE New Deaths: {rmse_deaths}")


In [ ]:
# Plot predictions for New Cases
plt.figure(figsize=(10, 6))
plt.plot(actual_cases.index, actual_cases, label='Actual New Cases')
plt.plot(actual_cases.index, final_cases_preds, label='Predicted New Cases (Hybrid)')
plt.title('Actual vs Predicted New Cases (Hybrid Model)')
plt.legend()
plt.show()

# Plot predictions for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(actual_deaths.index, actual_deaths, label='Actual New Deaths')
plt.plot(actual_deaths.index, final_deaths_preds, label='Predicted New Deaths (Hybrid Model)')
plt.title('Actual vs Predicted New Deaths (Hybrid Model)')
plt.legend()
plt.show()

**LightGBM**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load and prepare the dataset
data = pd.read_csv('/kaggle/input/final-dengu-journal-work/Dengue_final_as_MPOX.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data.set_index('Date', inplace=True)

# Feature engineering
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['week_of_year'] = data.index.isocalendar().week
data['lag_1_cases'] = data['New_Cases'].shift(1).fillna(0)
data['lag_2_cases'] = data['New_Cases'].shift(2).fillna(0)
data['lag_1_deaths'] = data['New_Deaths'].shift(1).fillna(0)
data['lag_2_deaths'] = data['New_Deaths'].shift(2).fillna(0)

# Train-test split using time-based cross-validation
X = data[['day_of_week', 'month', 'week_of_year', 'lag_1_cases', 'lag_2_cases', 'lag_1_deaths', 'lag_2_deaths']]
y_cases = data['New_Cases']
y_deaths = data['New_Deaths']

# TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the LightGBM model for cases and deaths
lgb_cases = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', n_jobs=-1)
lgb_deaths = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', n_jobs=-1)

# Define parameter grid for LightGBM
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

# Perform grid search with cross-validation
grid_cases = GridSearchCV(lgb_cases, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
grid_cases.fit(X, y_cases)

grid_deaths = GridSearchCV(lgb_deaths, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
grid_deaths.fit(X, y_deaths)

# Get the best models
best_lgb_cases = grid_cases.best_estimator_
best_lgb_deaths = grid_deaths.best_estimator_

In [ ]:
# Make predictions on the test set
pred_cases = best_lgb_cases.predict(X)
pred_deaths = best_lgb_deaths.predict(X)

# Evaluate model performance
rmse_cases = np.sqrt(mean_squared_error(y_cases, pred_cases))
rmse_deaths = np.sqrt(mean_squared_error(y_deaths, pred_deaths))
mae_cases = mean_absolute_error(y_cases, pred_cases)
mae_deaths = mean_absolute_error(y_deaths, pred_deaths)
r2_cases = r2_score(y_cases, pred_cases)
r2_deaths = r2_score(y_deaths, pred_deaths)

# Print results
print(f"LightGBM New Cases - RMSE: {rmse_cases}, MAE: {mae_cases}, R-squared: {r2_cases}")
print(f"LightGBM New Deaths - RMSE: {rmse_deaths}, MAE: {mae_deaths}, R-squared: {r2_deaths}")


In [ ]:
# Plotting the forecast and actuals for New Cases
plt.figure(figsize=(10, 6))
plt.plot(data.index, y_cases, label='Actual New Cases')
plt.plot(data.index, pred_cases, label='Predicted New Cases (LightGBM)')
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.title('Actual vs Predicted New Cases (LightGBM)')
plt.legend()
plt.show()

# Plotting the forecast and actuals for New Deaths
plt.figure(figsize=(10, 6))
plt.plot(data.index, y_deaths, label='Actual New Deaths')
plt.plot(data.index, pred_deaths, label='Predicted New Deaths (LightGBM)')
plt.xlabel('Date')
plt.ylabel('New Deaths')
plt.title('Actual vs Predicted New Deaths (LightGBM)')
plt.legend()
plt.show()


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score  # Import cross_val_score for cross-validation

# Objective function for Bayesian optimization with LightGBM
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 30)
    }
    
    # Initialize the model with suggested parameters
    model = lgb.LGBMRegressor(**params)
    
    # Perform time-series cross-validation and calculate the score
    score = -np.mean(cross_val_score(model, X, y_cases, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1))
    
    return score

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params
print("Best parameters:", best_params)

# Train the best model with optimized hyperparameters
best_lgb_cases = lgb.LGBMRegressor(**best_params)
best_lgb_cases.fit(X, y_cases)

best_lgb_deaths = lgb.LGBMRegressor(**best_params)
best_lgb_deaths.fit(X, y_deaths)


In [ ]:
!pip install pytorch-lightning pytorch-forecasting


In [ ]:
!pip install pytorch-forecasting pytorch-lightning

In [ ]:
!pip install pytorch-forecasting pytorch-lightning Metric

In [ ]:
import pytorch_lightning as pl  # Import the PyTorch Lightning module
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# Now we define the LightningModule explicitly
class TFTLightningModel(pl.LightningModule):
    def __init__(self, tft):
        super().__init__()
        self.model = tft
        self.save_hyperparameters(ignore=['tft', 'loss'])  # Ignore 'tft' and 'loss' during checkpointing

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x)  # Forward pass
        if isinstance(y_pred, tuple):
            y_pred = y_pred[0]  # If output is a tuple, extract the predictions
        loss = self.model.loss(y_pred, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x)  # Forward pass
        if isinstance(y_pred, tuple):
            y_pred = y_pred[0]  # If output is a tuple, extract the predictions
        loss = self.model.loss(y_pred, y)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.model.hparams.learning_rate)

# Initialize the LightningModule with the TemporalFusionTransformer
tft_lightning = TFTLightningModel(tft)

# Update Trainer configuration
trainer = pl.Trainer(
    max_epochs=30,
    devices=1,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=10,
    callbacks=[pl.callbacks.ModelCheckpoint(save_weights_only=True)]
)

# Train the model
trainer.fit(tft_lightning, train_dataloader, val_dataloader)

# Make predictions
predictions = tft.predict(val_dataloader, mode="prediction")

# Check the shape of predictions
print(f"Shape of predictions: {predictions.shape}")

# Extract the relevant part of predictions
if predictions.ndim > 2:
    predictions = predictions[:, :, 0]

# Reshape predictions to match scaler input
predictions = predictions[:, 0].reshape(-1, 1)

# Inverse scaling to get original values
predictions_inverse = scaler.inverse_transform(predictions)

# Perform inverse transform for validation targets
val_targets = torch.cat([y for x, y in iter(val_dataloader)], dim=0).cpu().numpy()
val_targets_inverse = scaler.inverse_transform(val_targets.reshape(-1, 1))

# Calculate RMSE
rmse = mean_squared_error(val_targets_inverse, predictions_inverse, squared=False)
print(f"RMSE: {rmse}")

# Plot results
plt.plot(val_targets_inverse, label="Actual")
plt.plot(predictions_inverse, label="Predicted")
plt.legend()
plt.title("Actual vs Predicted Cases")
plt.show()


In [ ]:
import pytorch_lightning as pl
from pytorch_forecasting import TemporalFusionTransformer, Baseline
from pytorch_forecasting.metrics import QuantileLoss

# Split data into train and validation sets
train_dataloader = tft_dataset.to_dataloader(train=True, batch_size=64, num_workers=0)
val_dataloader = tft_dataset.to_dataloader(train=False, batch_size=64, num_workers=0)

# Define the TFT model
tft = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    learning_rate=0.03,
    hidden_size=32,  # model size parameter
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Train the model
trainer = pl.Trainer(
    max_epochs=30,
    gpus=0,  # set to 1 if using GPU
)
trainer.fit(tft, train_dataloader, val_dataloader)


In [ ]:
# Predict on validation set
predictions = tft.predict(val_dataloader, mode="prediction")

# Inverse scaling to get original values
predictions = scaler.inverse_transform(predictions)
actuals = scaler.inverse_transform(val_dataloader.dataset.target_scaled)

# Calculate RMSE
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(actuals, predictions))
print(f"TFT RMSE: {rmse}")
